In [32]:
from copy import deepcopy
"""
Main structure 
- The board is represented by a dictionary which takes 
    the piece's coordinate as a key and stores MiniMaxNode

"""

class MiniMaxNode:
    """
    coord: coordinate of the piece
    color: either None, black(0) or white(1)
    pruned: used in Alpha-Beta pruning 
    """
    def __init__(self,coord,color=None,parent=None, chidren = None):
        self.coord = tuple(coord)
        self.color = color
        self.x = self.coord[0]
        self.y = self.coord[1]
        self.val = 0
        self.pruned = False

    def __str__(self):
        return (str(self.coord))


class Board:
    '''
    encodes the rule and board condition
    
    
    array: dictinary that encodes the color of each piece. black = 1, white = 0
    
    functions: 
        update: updates the board given new move and end nodes of the new move. 
            end nodes refer to the nodes which the new move forms the
            othello sandwich withsandwich with
    '''
    def __init__(self):
        #black = 0, white = 1. black goes first
        self.players = [Player(0), Player(1)]
        self.player = self.players[0]
        #Initializing an empty board
        self.array = {}
        for x in range(8):
            for y in range(8):
                self.array[str(x)+','+str(y)]=MiniMaxNode((x,y))

        #Initializing center values
        self.array['3,3']=MiniMaxNode((3,3), color = 1)
        self.array['3,4']=MiniMaxNode((3,4), color = 0)
        self.array['4,3']=MiniMaxNode((4,3), color = 0)
        self.array['4,4']=MiniMaxNode((4,4), color = 1)

    
    def update(self, new_move, ends):
        '''
        input: MiniMaxnode of new move, all ends that opponent's othellos are sandwiched with
        updates the board based on othello rule 
        all opponent piece that are sandwiched by new piece will turn to player's color
        switch the player 
        '''
        #update the new node's color
        new_move.color = self.player.color
        for end in ends:
            end.color = self.player.color
            #end nodes on the same row
            if new_move.x == int(end.x):
                row = new_move.x
                #pieces to the right change color
                if end.y-new_move.y > 0:
                    for column in range(new_move.y+1, end.y):
                        self.array[str(row)+','+str(column)].color = self.player.color
                else: 
                    for column in range(end.y+1, new_move.y):
                        self.array[str(row)+','+str(column)].color = self.player.color
            #end nodes on same column
            if new_move.y == end.y:
                column = new_move.y
                if end.x-new_move.x > 0 :
                    for row in range(new_move.x+1, end.x):
                        self.array[str(row)+','+str(column)].color = self.player.color
                else: 
                    for row in range(end.x+1, new_move.x):
                        self.array[str(row)+','+str(column)].color = self.player.color
            #end nodes diagonal
            else:
                row = new_move.x 
                column = new_move.y
                if end.x-new_move.x > 0 and end.y-new_move.y>0:
                    for i in range(new_move.x+1, (end.x)):
                        row += 1
                        column += 1
                        self.array[str(row)+','+str(column)].color = self.player.color
                if end.x-new_move.x > 0 and end.y-new_move.y<0:
                    for i in range(new_move.x+1, (end.x)):
                        row += 1
                        column -= 1
                        self.array[str(row)+','+str(column)].color = self.player.color
                if end.x-new_move.x < 0 and end.y-new_move.y>0:
                    for i in range(new_move.y+1, (end.y)):
                        row-= 1
                        column += 1
                        self.array[str(row)+','+str(column)].color = self.player.color
                else: 
                    for i in range(end.x+1, (new_move.x)):
                        row-= 1
                        column -= 1
                        self.array[str(row)+','+str(column)].color = self.player.color
        #switch player
        if self.player == self.players[0]:
            self.player = self.players[1]
        else:
            self.player = self.players[0]
        
class Player: 
    '''
    won: whether the player won 
    pass: whether the player pass the turn 
    move: new move when it's the player's turn 
    '''
    def __init__(self, color):
        self.color = color
        self.move = []
        self.won = False
        self.must_pass = False
    
    def next_moves(self, board): 
        """
        input: board
        approach:
        +/- 1 in x/y of every opponent pieces
        output: returns a list of possible next moves
        """
        moves = []
        for key in board.array:
            node = board.array[key]
            if is_valid(node, board):
                #if the node is empty and valid
                moves.append(node)
        if moves == []:
            return None
        return moves
    def __str__(self):
        return str(self.color)


In [33]:
def MiniMax(board):
    '''
    returns the mini-maximmizing node
    '''
    moves = []
    Max = -float('inf')
    Min = float('inf')
    

    if board.player.next_moves == None:
        return None
    for old_node in board.player.next_moves(board):
        coord = str(old_node.coord[0])+','+str(old_node.coord[1])
        save_board = deepcopy(board)
        #create a new board so updates in players do not interfere
        #with the game itself
        node = save_board.array[coord]
        #calculate utility for all possible move for 
        #computer
        val = utility(node, save_board)
        save_board.update(node, find_end(node,save_board))
        #moves a player can play for a computer's move
        children = save_board.player.next_moves(save_board)
        if children == None: 
            Max_node = None
            continue
        for index in range(len(children)):
            #calculate local minimum and prune nodes as needed
            local_min = float('inf')
            #alpha-beta pruning
            #if the utility of next player's move was less than 
            #the current minimum, prune rest of the nodes
            if -utility(children[index],save_board)+val < local_min:
                local_min = children[index]
            if -utility(children[index],save_board)+val< Max:
                for node in children[index+1:]:
                    node.pruned = True
                    break
        if -utility(local_min,save_board)+val > Max:
            Max = -utility(local_min,save_board)+val
            Max_node = old_node
    #if none of them have children, take the move with highest utility
    if Max_node == None:
        max_utility = max(utility(node, board) for node in board.player.next_moves(board))
        for node in board.player.next_moves(board):
            if utility(node,board) == max_utility:
                Max_node = node
            

    return Max_node

def utility(node, board):
    ends = find_end(node, board)
    utility = 0
    for end in ends:
        #end nodes on same row
        if node.x == end.x:
            utility += abs(node.y-end.y)

        #end nodes diagonal or same column
        else:
            utility+= abs(end.x - node.x)
    return utility
    
def terminate(board):
    """
    checks for termination condition
    """
    for key in board.array:
        if board.array[key].color == None: 
        #if all grids are filled, return true 
            return False
    if must_pass(board) and must_pass(deepcopy(board).update(None, None)):
        
        return True
    return True
def must_pass(board):
    if board.player.next_moves(board) == []:
        return True 
    return False
    

def find_end(move,board):
    '''
    finds the end of the othello sandwich, given the new piece input
    output: lists of MiniMax nodes that are end nodes of othello sandwich
    '''
    ends = []
    row = move.x
    column = move.y
    
    directions = [[1,0],[0,1],[0,-1],[-1,0],[1,-1],[-1,1],[1,1],[-1,-1]]
    for direction in directions: 
        search_row = row
        search_column = column
        while 0<=search_row<8 and 0<=search_column<8:
            search_row += direction[0]
            search_column += direction[1]
            if search_row == 8 or search_column == 8 or search_row <0 or search_column <0:
                break 
            if board.array[str(search_row)+','+str(search_column)].color == None:
                break 
            if board.array[str(search_row)+','+str(search_column)].color == board.player.color and (abs(search_row-row)==1 or abs(search_column-column)==1):
                break
            if board.array[str(search_row)+','+str(search_column)].color == board.player.color and (abs(search_row-row)>1 or abs(search_column-column)>1):
                ends.append(board.array[str(search_row)+','+str(search_column)]) 
                break
            if board.array[str(search_row)+','+str(search_column)].color == abs(board.player.color-1):
                continue
    return ends
    

def is_valid(move, board):
    #if none of the adjacent nodes are of the other player, return false
    if move.color == None and find_end(move, board) != []:
        return True 
    return False 

def score(board):
    w = 0
    b = 0
    for key in board.array:
        node = board.array[key]
        if node.color == 0:
            b+=1
        if node.color == 1:
            w += 1
    return [b, w]
        
"""
extra code to test MiniMax function 
    def MiniMax_(board):
    '''
    returns the mini-maximmizing node
    '''
    moves = []
    Max = -float('inf')
    Min = float('inf')
    

    if board.player.next_moves == None:
        return None
    for old_node in board.player.next_moves(board):
        print('Possible next move for computer is ', old_node)
    for old_node in board.player.next_moves(board):
        coord = str(old_node.coord[0])+','+str(old_node.coord[1])
        save_board = deepcopy(board)
        #create a new board so updates in players do not interfere
        #with the game itself
        node = save_board.array[coord]
        #calculate utility for all possible move for 
        #computer
        val = utility(node, save_board)
        save_board.update(node, find_end(node,save_board))
        #moves a player can play for a computer's move
        children = save_board.player.next_moves(save_board)
        if children == None: 
            Max_node = None
            continue
        for index in range(len(children)):
            #calculate local minimum and prune nodes as needed
            print('node', children[index],'is a child node of',old_node)
            local_min = float('inf')
            #alpha-beta pruning
            #if the utility of next player's move was less than 
            #the current minimum, prune rest of the nodes
            if -utility(children[index],save_board)+val < local_min:
                local_min = children[index]
            if -utility(children[index],save_board)+val< Max:
                for node in children[index+1:]:
                    print(node,'with utility',-utility(children[index],save_board)+val,'pruned because it is less than', Max_node,'with utility',utility(node,board)+val)
                    node.pruned = True
                    break
        if -utility(local_min,save_board)+val > Max:
            Max = -utility(local_min,save_board)+val
            Max_node = old_node
    #if none of them have children, take the move with highest utility
    if Max_node == None:
        max_utility = max(utility(node, board) for node in board.player.next_moves(board))
        for node in board.player.next_moves(board):
            if utility(node,board) == max_utility:
                Max_node = node
    return Max_node"""
    
    
def play(board):
    while terminate(board) == False:
        print('player is', board.player)
        if board.player.color == 0:
            print('It is computer1 turn!')
            #ask for input if the player if human
            #if player cannot play(no next move), then skip 
            if board.player.next_moves(board) == None:
                #if the computer has no move
                board.player.must_pass == True
                board.player = board.players[0]
                print('Damn, I pass.')
                break
            else:
                move = MiniMax(board) 
            print('my move is',move)
        print('Player',board.player, 'played', move)
            
        if board.player.color == 1: 
            print('It is computer2 turn!')
            #computer is a maximizer
            if board.player.next_moves(board) == None:
                #if the computer has no move
                board.player.must_pass == True
                board.player = board.players[0]
                print('Damn, I pass.')
                break
            else:
                move = MiniMax(board) 
            print('my move is',move)
        print('Computer',board.player.color + 1, 'played', move)
        board.update(move, find_end(move, board))
        #update the board
        visualize(board)
    if terminate(board) == True:
        
        if score(board)[0] > score(board)[1]:
            print('Computer 1 win!')
        if score(board)[1] > score(board)[0]:
            print('Computer 2 win!')
        else: 
            print('It is a draw! Good game :))') 
        

In [34]:
othello_board = Board()
def visualize(board):
    print("   0    1    2    3    4    5    6    7  ")
    print(" ----" * 8)
    for row in range(8):
        print_row = []
        state= board.array
        for column in range(8): 
            item = state[str(row)+','+str(column)]
            if item.color is not None:
                if item.color == 0:
                    print_row.append("| ●  ")
                if item.color == 1:
                    print_row.append("| ○  ")
            else:
                print_row.append("|    ")    
        print(row,print_row[0]+print_row[1]+print_row[2]+print_row[3]+print_row[4]+print_row[5]+print_row[6]+print_row[7]+'|')
        print(" ----" * 8)
    print('score: human-', score(board)[0], 'me:', score(board)[1])
visualize(othello_board)
play(othello_board)



   0    1    2    3    4    5    6    7  
 ---- ---- ---- ---- ---- ---- ---- ----
0 |    |    |    |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
1 |    |    |    |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
2 |    |    |    |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
3 |    |    |    | ○  | ●  |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
4 |    |    |    | ●  | ○  |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
5 |    |    |    |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
6 |    |    |    |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
7 |    |    |    |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
score: human- 2 me: 2
player is 0
It is computer1 turn!
my move is (2, 3)
Player 0 played (2, 3)
Computer 1 played (2, 3)
   0    1    2    3    4    5    6    7  
 ---- ---- ---- ---- ---- ---- ---- ----
0 |    |    |    |    |    |    

my move is (5, 2)
Player 0 played (5, 2)
Computer 1 played (5, 2)
   0    1    2    3    4    5    6    7  
 ---- ---- ---- ---- ---- ---- ---- ----
0 | ○  | ○  | ○  |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
1 |    | ○  | ○  |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
2 |    | ○  | ○  | ●  | ○  |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
3 | ●  | ○  | ●  | ○  | ●  |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
4 |    | ●  | ●  | ●  | ●  | ●  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
5 |    | ○  | ●  |    |    | ●  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
6 |    |    |    |    |    | ●  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
7 |    |    |    |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
score: human- 12 me: 11
player is 1
Player 1 played (5, 2)
It is computer2 turn!
my move is (5, 4)
Computer 2 played (5, 4)
   0    1    2    3    4    5    6    7  
 ----

my move is (5, 0)
Player 0 played (5, 0)
Computer 1 played (5, 0)
   0    1    2    3    4    5    6    7  
 ---- ---- ---- ---- ---- ---- ---- ----
0 | ○  | ○  | ○  |    |    |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
1 | ○  | ○  | ○  |    | ●  |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
2 | ○  | ○  | ○  | ●  | ●  |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
3 | ○  | ○  | ●  | ●  | ○  |    |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
4 | ○  | ●  | ○  | ○  | ○  | ○  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
5 | ●  | ●  | ●  | ●  | ○  | ○  | ○  |    |
 ---- ---- ---- ---- ---- ---- ---- ----
6 |    |    | ●  |    |    | ○  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
7 |    |    |    |    |    |    | ○  |    |
 ---- ---- ---- ---- ---- ---- ---- ----
score: human- 11 me: 22
player is 1
Player 1 played (5, 0)
It is computer2 turn!
my move is (6, 1)
Computer 2 played (6, 1)
   0    1    2    3    4    5    6    7  
 ----

my move is (4, 7)
Computer 2 played (4, 7)
   0    1    2    3    4    5    6    7  
 ---- ---- ---- ---- ---- ---- ---- ----
0 | ○  | ○  | ○  | ○  | ○  | ○  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
1 | ○  | ○  | ○  | ○  | ○  | ○  | ○  | ○  |
 ---- ---- ---- ---- ---- ---- ---- ----
2 | ○  | ○  | ○  | ●  | ○  | ○  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
3 | ○  | ○  | ●  | ●  | ●  | ○  | ○  |    |
 ---- ---- ---- ---- ---- ---- ---- ----
4 | ○  | ○  | ●  | ○  | ●  | ●  |    | ○  |
 ---- ---- ---- ---- ---- ---- ---- ----
5 | ●  | ○  | ●  | ○  | ●  | ●  | ○  | ●  |
 ---- ---- ---- ---- ---- ---- ---- ----
6 |    | ●  | ●  | ●  |    | ○  |    |    |
 ---- ---- ---- ---- ---- ---- ---- ----
7 |    |    | ●  |    |    |    | ○  |    |
 ---- ---- ---- ---- ---- ---- ---- ----
score: human- 16 me: 32
player is 0
It is computer1 turn!
my move is (2, 6)
Player 0 played (2, 6)
Computer 1 played (2, 6)
   0    1    2    3    4    5    6    7  
 ---- ---- ---- ---- ---- --

In [16]:
def MiniMax_(board):
    '''
    returns the mini-maximmizing node
    '''
    moves = []
    Max = -float('inf')
    Min = float('inf')
    

    if board.player.next_moves == None:
        return None
    for old_node in board.player.next_moves(board):
        coord = str(old_node.coord[0])+','+str(old_node.coord[1])
        save_board = deepcopy(board)
        #create a new board so updates in players do not interfere
        #with the game itself
        node = save_board.array[coord]
        #calculate utility for all possible move for 
        #computer
        val = utility(node, save_board)
        save_board.update(node, find_end(node,save_board))
        #moves a player can play for a computer's move
        children = save_board.player.next_moves(save_board)
        if children == None: 
            Max_node = None
            continue
        for index in range(len(children)):
            #calculate local minimum and prune nodes as needed
            local_min = float('inf')
            #alpha-beta pruning
            #if the utility of next player's move was less than 
            #the current minimum, prune rest of the nodes
            if -utility(children[index],save_board)+val < local_min:
                local_min = children[index]
            if -utility(children[index],save_board)+val< Max:
                for node in children[index+1:]:
                    print(node,'pruned. Due to node', children[index],'with utility',-utility(children[index],save_board)+val)
                    node.pruned = True
                    break
        if -utility(local_min,save_board)+val > Max:
            Max = -utility(local_min,save_board)+val
            Max_node = old_node
    #if none of them have children, take the move with highest utility
    if Max_node == None:
        max_utility = max(utility(node, board) for node in board.player.next_moves(board))
        for node in board.player.next_moves(board):
            if utility(node,board) == max_utility:
                Max_node = node
    return Max_node

In [17]:
othello_board = Board()
print(MiniMax_(othello_board))

(2, 3)None
